<h1> Assignment 7

<h1> Name & Z-ID 
<h3> Daniel Bendik
<h3> z1938845
<h3> CSCI 490
<h3> Dr. Maoyuan Sun

<h1> Download & Extract Files

In [10]:
import aiohttp
import asyncio
import os
import zipfile
from pathlib import Path

async def download_file(url, filename):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            with open(filename, 'wb') as f:
                while True:
                    chunk = await response.content.read(1024)
                    if not chunk:
                        break
                    f.write(chunk)
    print(f'Downloaded {filename}')

async def download_all_files():
    file_url = 'https://faculty.cs.niu.edu/~smaoyuan/courses/csci503/a7/'
    ranges = [(75,79), (80,99), (100,119), (120,199), (200,249), (250,999)]

    tasks = []

    for start, end in ranges:
        filename = f'{start}_million_to_{end}_million.zip'
        url = f'{file_url}{filename}'
        tasks.append(asyncio.create_task(download_file(url, filename)))

    await asyncio.gather(*tasks)

async def extract_zip_files():
    zip_files = [f for f in os.listdir() if f.endswith('.zip')]
    extract_dir = 'artist-data'
    Path(extract_dir).mkdir(parents=True, exist_ok=True)

    for file in zip_files:
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

async def main():
    await download_all_files()
    await extract_zip_files()

if __name__ == '__main__':
    if asyncio.get_event_loop().is_running():
        await main()

Downloaded 120_million_to_199_million.zip
Downloaded 80_million_to_99_million.zip
Downloaded 100_million_to_119_million.zip
Downloaded 75_million_to_79_million.zip
Downloaded 250_million_to_999_million.zip
Downloaded 200_million_to_249_million.zip


<h1> Find Matching Files

In [17]:
from pathlib import Path

def get_files(directory):
    npy_files = []
    for file in Path(directory).rglob('*.npy'):
        npy_files.append(file)
    return npy_files

npy_files = get_files('artist-data')

x = 1
for file in npy_files:  # Print the paths to each file
    print(x, file)
    x+=1

1 artist-data/75_million_to_79_million/United States/kenny-g.npy
2 artist-data/75_million_to_79_million/United States/alabama.npy
3 artist-data/75_million_to_79_million/United States/luke-bryan.npy
4 artist-data/75_million_to_79_million/United States/imagine-dragons.npy
5 artist-data/75_million_to_79_million/United States/gloria-estefan.npy
6 artist-data/75_million_to_79_million/Other/the-weeknd.npy
7 artist-data/100_million_to_119_million/United States/george-strait.npy
8 artist-data/100_million_to_119_million/United States/janet-jackson.npy
9 artist-data/100_million_to_119_million/United Kingdom/george-michael.npy
10 artist-data/100_million_to_119_million/Other/nicki-minaj.npy
11 artist-data/80_million_to_99_million/United States/lionel-richie.npy
12 artist-data/80_million_to_99_million/United States/post-malone.npy
13 artist-data/80_million_to_99_million/United States/ariana-grande.npy
14 artist-data/80_million_to_99_million/United States/flo-rida.npy
15 artist-data/80_million_to_99

<h1> Use Threads to Process Files

In [25]:
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

def get_files(directory):  # Reused function from Part 2 "Find Matching Files"
    npy_files = []
    for file in Path(directory).rglob('*.npy'):
        npy_files.append(file)
    return npy_files

def process_npy_file(file_path):
    # Convert the Numpy array to dataframe
    arr = np.load(file_path)
    df = pd.DataFrame({'Date': arr[0], 'Views': arr[1]})

    # Convert the filename into the artist's name
    artist_name = file_path.stem.replace('-', ' ').title()
    df['Artist'] = artist_name
    return df

async def main():
    npy_files = get_files('artist-data')
    with ThreadPoolExecutor() as executor:
        dfs = list(executor.map(process_npy_file, npy_files))

    # Concatenate dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    # Split the combined dataframes into monthly dataframes
    for month, month_df in combined_df.groupby(combined_df['Date']):
        output_filename = f'{month}.csv.gz'
        month_df.to_csv(output_filename, index=False, compression='gzip')  # Save them as .csv.gz files

if __name__ == '__main__':
    if asyncio.get_event_loop().is_running():  # Do this since we're using JupyterLab
        await main()

<h2> ^ This takes about 10 seconds to finish executing ^